<a href="https://colab.research.google.com/github/lnpu/firstNeuralNetwork/blob/master/SecondNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
from scipy import special, ndimage
import numpy as np
import matplotlib.pyplot as plot

debug = False

def _print(a):
  if(debug):
    print(a)

In [0]:
# layer structure
class Layer:
  def __init__(self, inputNodes, outputNodes, learningRate):
    self.iNode = inputNodes
    self.oNode = outputNodes
    self.lr = learningRate
    self.activation_function = lambda x:special.expit(x)
    self.weight = np.random.normal(0.0,pow(self.oNode,-0.5),(self.oNode, self.iNode)) #(np.random.rand(self.hNode, self.iNode)-0.5)
    _print(self.weight.shape)
    pass

In [56]:

# neural network structure
class NN:
  # construct neural work struction
  def __init__(self):
    self.layers = []
    pass
  # using activation function iterate network once
  def train(self, input_list, target_list, epochs=1):
    _print('training start')
    for epoch in range(epochs):
      _print("start epoch %d"%epoch)
      inputs = np.array(input_list, ndmin=2, copy=True).T
      _print("inputs shape is %s"%(str(inputs.shape)))
      targets = np.array(target_list, ndmin=2, copy=True).T
      _print("targets shape is %s"%str(targets.shape))
      # -> using forward pass compute params 
      for index,layer in enumerate(self.layers):
        _print("forward pass %d ..."%index)
        _print("weight %d shape is %s"%(index, str(self.layers[index].weight.shape)))
        self.layers[index].inputs = np.dot(self.layers[index].weight, inputs)
        # outputs is an input that calculated from activation function
        self.layers[index].outputs = self.layers[index].activation_function(self.layers[index].inputs)
        _print(self.layers[index].outputs.shape)
        inputs = self.layers[index].outputs
      # -> use backword pass to update weight
      for index,layer in enumerate(self.layers):
        # stating at final layer
        t = len(self.layers) - 1 - index
        #_print("backpass pass %d ..."%t)
        # if output layer, using targets - outputs to update errors
        if index == 0:
          # calculate final layer loss
          self.layers[t].errors = targets - self.layers[t].outputs
          # calculate and update final layer weight
          x = (self.layers[t].errors*self.layers[t].outputs*(1.0-self.layers[t].outputs))
          xx = np.dot(x, np.transpose(self.layers[t-1].outputs))
          #print(self.layers[t].weight.shape, self.layers[t].lr, x.shape, xx.shape )
          self.layers[t].weight += self.layers[t].lr * xx
        # if input layer
        elif t == 0:
          #self.layers[t].errors = np.dot(self.layers[t+1].weight.T, self.layers[t+1].errors)
          #x = (self.layers[t].errors*self.layers[t].outputs*(1.0-self.layers[t].outputs))
          #xx = np.dot(x, np.transpose(self.layers[t].outputs))
          #print(self.layers[t].weight.shape, self.layers[t].lr, x.shape, xx.shape )
          #self.layers[t].weight += self.layers[t].lr * xx
          pass
        # if hidden layer, using next layer to update errors
        else:
          self.layers[t].errors = np.dot(self.layers[t+1].weight.T, self.layers[t+1].errors)
          x = (self.layers[t].errors*self.layers[t].outputs*(1.0-self.layers[t].outputs))
          xx = np.dot(x, np.transpose(self.layers[t].outputs))
          #print(self.layers[t].weight.shape, self.layers[t].lr,x.shape, xx.shape )
          self.layers[t].weight += self.layers[t].lr * xx
        pass
    pass
  def add_layer(self, layer):
    self.layers.append(layer)
    pass
  # predict an input using trained network
  def predict(self, inputs_list):
    inputs = np.array(inputs_list, ndmin = 2).T
    for index,layer in enumerate(self.layers):
      self.layers[index].inputs = np.dot(self.layers[index].weight, inputs)
      self.layers[index].outputs = self.layers[index].activation_function(self.layers[index].inputs)
      inputs = self.layers[index].outputs
    return np.argmax(self.layers[len(self.layers)-1].outputs)


data_file = open('/content/sample_data/mnist_test.csv', 'r')
data_list = data_file.readlines()
data_file.close()
# training neural network
iNode = 784
oNode = 10
hNode = 100
nn = NN()
nn.add_layer(Layer(iNode, 100, 0.3))
nn.add_layer(Layer(100, 100,   0.3))
nn.add_layer(Layer(100, 100,   0.3))
nn.add_layer(Layer(100, 100,   0.3))
nn.add_layer(Layer(100, 100,   0.3))
nn.add_layer(Layer(100, 100,   0.3))
nn.add_layer(Layer(100, oNode, 0.3))
for line in data_list:
  all_values = line.split(',')
  correct_label = int(all_values[0])
  inputs = (np.asfarray(all_values[1:])/255.0*0.99 + 0.01) #.reshape(28,28)
  targets = np.zeros(oNode)+0.01
  targets[int(all_values[0])] = 0.99
  nn.train(inputs, targets)
  pass
print('training finished')

training finished


In [57]:
# testing nn
_tfile = open('/content/sample_data/mnist_test.csv')
data_list = _tfile.readlines()
_tfile.close()
# test neural network use scorecard
scorecard = []
for line in data_list:
  all_values = line.split(',')
  correct_label = int(all_values[0])
  inputs = (np.asfarray(all_values[1:])/255.0*0.99 + 0.01) #.reshape(28,28)
  outputs = nn.predict(inputs)
  label = np.argmax(outputs)
  #print("real: %s, predict: %s\n"%(correct_label, label))
  scorecard.append(1 if label==correct_label else 0)
  pass
print("Accuracy : %f , testing data size : %d"%(len([i for i in scorecard if i>0])/len(scorecard), len(data_list)))

# testing nn
_tfile = open('/content/sample_data/mnist_test.csv')
data_list = _tfile.readlines()
_tfile.close()
all_values = data_list[44].split(',')
r = int(all_values[0])
p = nn.predict((np.asfarray(all_values[1:])/255*0.99)+0.01)
print("real %d , predict %d"%(r,p))

Accuracy : 0.098000 , testing data size : 10000
real 3 , predict 6
